In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

## **Utility functions**

In [2]:
def sphere_to_cartesian(r, gamma, theta):
    '''Convert spherical coordinates to cartesian
       r - sphere radius
       gamma - polar angle
       theta - azimuth angle
       return - cartesian coordinates
    '''
    return (r * np.sin(gamma) * np.sin(theta),
            r * np.sin(gamma) * np.cos(theta),
            r * np.cos(gamma))

def s_arc(sr, c_gamma, c_theta, r_delta, start, end, n=32):
    '''Get arc points plotted on a sphere's surface
       sr - sphere radius
       c_gamma - polar angle of the circle's center
       c_theta - azimuth angle of the circle's center
       r_delta - angle between OC and OP, where O is sphere's center,
                 C is the circle's center, P is any point on the circle
       start - arc start angle
       end - arc end angle
       n - number of points
       return - circle points
    '''
    t = np.expand_dims(np.linspace(start, end, n), axis=1)
    a = sphere_to_cartesian(1.0, c_gamma + r_delta, c_theta)
    k = sphere_to_cartesian(1.0, c_gamma, c_theta)
    c = np.cos(t) * a + np.sin(t) * np.cross(k, a) + \
        np.dot(k, a) * (1.0 - np.cos(t)) * k
    c = c * sr
    return [dim.squeeze() for dim in np.hsplit(c, 3)]

def s_inv(gamma0, gamma):
    phi = np.arccos(np.tan(gamma0) / np.tan(gamma))    
    return np.arccos(np.cos(gamma) / np.cos(gamma0)) / np.sin(gamma0) - phi


def circle3d_by3points(a, b, c):
    u = b - a
    w = np.cross(c - a, u)
    u = u / np.linalg.norm(u)
    w = w / np.linalg.norm(w)
    v = np.cross(w, u)
    
    bx = np.dot(b - a, u)
    cx, cy = np.dot(c - a, u), np.dot(c - a, v)
    
    h = ((cx - bx / 2.0) ** 2 + cy ** 2 - (bx / 2.0) ** 2) / (2.0 * cy)
    cc = a + u * (bx / 2.0) + v * h
    r = np.linalg.norm(a - cc)

    return r, cc


def rotation_matrix(axis, alpha):
    ux, uy, uz = axis
    sina, cosa = np.sin(alpha), np.cos(alpha)
    r_mat = np.array((
        (cosa + (1.0 - cosa) * ux ** 2,
         ux * uy * (1.0 - cosa) - uz * sina,
         ux * uz * (1.0 - cosa) + uy * sina),
        (uy * ux * (1.0 - cosa) + uz * sina,
         cosa + (1.0 - cosa) * uy ** 2,
         uy * uz * (1.0 - cosa) - ux * sina),
        (uz * ux * (1.0 - cosa) - uy * sina,
         uz * uy * (1.0 - cosa) + ux * sina,
         cosa + (1.0 - cosa) * uz ** 2),
    ))

    return r_mat


def angle_between(o, a, b):
    p = a - o
    q = b - o
    return np.arccos(np.dot(p, q) / (np.linalg.norm(p) * np.linalg.norm(q)))


def project_to_xy_from_sphere_center(pts, sphere_r):
    gammas = np.arccos(np.dot(pts, np.array((0.0, 0.0, 1.0))) / \
                       np.linalg.norm(pts, axis=1))
    thetas = np.arctan2(pts[:, 0], pts[:, 1])
    radiuses = sphere_r /  np.cos(gammas)
    proj_pts = np.dstack(sphere_to_cartesian(radiuses,
                                             gammas, thetas)).squeeze()
    
    return proj_pts


## **Bevel gear**

In [3]:
class BevelGear:    
    def __init__(self, module=1.0, teeth_number=18,
                 cone_angle=45.0, face_width=4.0, bore=6.0,
                 pressure_angle=20.0, helix_angle=0.0,
                 addendum_cf=1.0, dedendum_cf=1.25, curve_points=16):
        m = module
        self.z = z = teeth_number
        self.curve_points = curve_points
        fw = face_width
        ka = addendum_cf
        kd = dedendum_cf
        alpha = np.radians(pressure_angle)
        
        # pitch cone angle
        self.gamma_p = gamma_p = np.radians(cone_angle)

        # base/pitch circle radius
        rp = m * z / 2.0
        
        # great sphere radius, also corresponds to pitch cone flank length
        self.gs_r = gs_r = rp / np.sin(gamma_p) 
        
        # angles of base, face and root cones 
        self.gamma_b = gamma_b = np.arcsin(np.cos(alpha) * np.sin(gamma_p))
        self.gamma_f = gamma_f = gamma_p + np.arctan(ka * m / gs_r)
        self.gamma_r = gamma_r = gamma_p - np.arctan(kd * m / gs_r)

        phi_r = s_inv(gamma_b, gamma_p);
        self.mirrpoint = mirrpoint = np.pi / z + 2.0 * phi_r
                
        self.tau = tau = np.pi * 2.0 / z
        gamma_tr = max(gamma_b, gamma_r)

        # Tooth left flank curve points
        gamma = np.linspace(gamma_tr, gamma_f, curve_points)
        theta = s_inv(gamma_b, gamma)
        self.tooth_lflank = np.dstack(sphere_to_cartesian(gs_r,
                                                          gamma,
                                                          theta)).squeeze()
        # Tooth tip curve points
        theta_tip = np.linspace(theta[-1], mirrpoint - theta[-1], curve_points)
        self.tooth_tip = np.dstack(sphere_to_cartesian(gs_r,
                                                       np.full(curve_points,
                                                               gamma_f),
                                                       theta_tip)).squeeze()
        # Tooth right flank curve points
        self.tooth_rflank = np.dstack(
                                sphere_to_cartesian(gs_r,
                                                    gamma[::-1],
                                                    mirrpoint - \
                                                        theta[::-1])).squeeze()

        # Tooth root curve points
        if gamma_r < gamma_b:
            p1 = self.tooth_rflank[-1]
            p2 = np.array(sphere_to_cartesian(gs_r, gamma_b, theta[0] + tau))
            p3 = np.array(sphere_to_cartesian(gs_r, gamma_r,
                                              (tau + mirrpoint) / 2.0))
            rr, rcc = circle3d_by3points(p1, p2, p3)
            rcc_gamma = np.arccos(np.dot(p3, rcc) / \
                                  (np.linalg.norm(p3) * np.linalg.norm(rcc)))
            p1p3 = angle_between(rcc, p1, p3)
            a_start = (np.pi - p1p3 * 2.0) / 2.0
            a_end = -a_start + np.pi
            self.tooth_root = np.dstack(s_arc(gs_r, gamma_r + rcc_gamma,
                                              (tau + mirrpoint) / 2.0,
                                              rcc_gamma,
                                              np.pi / 2.0 + a_start,
                                              np.pi / 2.0 + a_end,
                                              curve_points)).squeeze()
        else:
            r_theta = np.linspace(mirrpoint - theta[0],
                                  theta[0] + tau, curve_points)
            self.tooth_root = np.dstack(sphere_to_cartesian(
                                                        gs_r,
                                                        np.full(curve_points,
                                                                gamma_tr),
                                                        r_theta)).squeeze()


    def plot(self, plt_axes, sphere=True, pitch_circle=False,
             base_circle=False, face_circle=False, root_circle=False,
             t_matrix=None):
        if t_matrix is None:
            t_matrix = np.identity(3)
        
        tooth_pts = np.vstack((self.tooth_lflank, self.tooth_tip,
                               self.tooth_rflank, self.tooth_root))
        gear_pts = np.concatenate([
                    tooth_pts @ rotation_matrix((0.0, 0.0, 1.0),
                                                self.tau * i)
                                   for i in range(self.z)]) @ t_matrix
        
        if sphere:
            u = np.linspace(0.0, 2.0 * np.pi, 24)
            v = np.linspace(0.0, np.pi, 24)
            gs_x = self.gs_r * np.outer(np.cos(u), np.sin(v))
            gs_y = self.gs_r * np.outer(np.sin(u), np.sin(v))
            gs_z = self.gs_r * np.outer(np.ones(np.size(u)), np.cos(v))
            ax.plot_surface(gs_x, gs_y, gs_z, color='#aaa', alpha=0.1)

        c_points = self.curve_points * 2
        t = np.linspace(0.0, np.pi * 2.0, c_points)
        circles = ((pitch_circle , self.gamma_p),
                   (base_circle, self.gamma_b),
                   (face_circle, self.gamma_f),
                   (root_circle, self.gamma_r))
        
        for do_plot, gamma in circles:
            if do_plot:
                pts = np.dstack(sphere_to_cartesian(
                                    np.full(c_points, self.gs_r),
                                            gamma, t)).squeeze()
                pts = pts @ t_matrix
                x, y, z = (ax.squeeze() for ax in np.hsplit(pts, 3))
                ax.plot3D(x, y, z, linestyle='dashed',
                          linewidth=0.5, alpha=0.8)
            
        pts_x, pts_y, pts_z = (ax.squeeze() for ax in np.hsplit(gear_pts, 3))
        plt_axes.plot3D(pts_x, pts_y, pts_z, linewidth=1.0)


In [4]:
class BevelGearPair:
    
    def __init__(self, module, gear_teeth, pinion_teeth, face_width,
                 axis_angle=90.0, gear_bore=3.0, pinion_bore=3.0,
                 pressure_angle=20.0, helix_angle=0.0, addendum_cf=1.0,
                 dedendum_cf=1.25, curve_points=16):
        
        self.axis_angle = axis_angle = np.radians(axis_angle)
        
        # Cone Angle of the Gear
        delta_gear = np.arctan(np.sin(axis_angle) / \
                               (pinion_teeth / gear_teeth + \
                                np.cos(axis_angle)))
        
        # Cone Angle of the Pinion
        delta_pinion = np.arctan(np.sin(axis_angle) / \
                               (gear_teeth / pinion_teeth + \
                                np.cos(axis_angle)))
        
        self.gear = BevelGear(module, gear_teeth, np.degrees(delta_gear),
                              face_width, gear_bore, pressure_angle,
                              helix_angle, addendum_cf, dedendum_cf,
                              curve_points)
        
        self.pinion = BevelGear(module, pinion_teeth, np.degrees(delta_pinion),
                                face_width, gear_bore, pressure_angle,
                                helix_angle, addendum_cf, dedendum_cf,
                                curve_points)


    def plot(self, plt_axes):
        
        g_shift_a = 0.0 if not (self.pinion.z % 2) else np.pi / self.gear.z
        g_mat = rotation_matrix((0.0, 0.0, 1.0),
                                -self.gear.mirrpoint / 2.0 + np.pi + np.pi / self.gear.z)
        p_mat = rotation_matrix((0.0, 0.0, 1.0),
                                -self.pinion.mirrpoint / 2.0)
        p_mat = p_mat @ rotation_matrix((1.0, 0.0, 0.0), -self.axis_angle)

        self.gear.plot(ax, True, True, True, True, True, g_mat)
        self.pinion.plot(ax, False, True, True, True, True, p_mat)

gears = BevelGearPair(module=1.0, gear_teeth=13, pinion_teeth=7, face_width=5.0, axis_angle=90.0)

In [5]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

gears.plot(ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …